# Scenario Analysis Example

This notebook demonstrates how to perform scenario analysis for health economic evaluation.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Define a scenario analysis function
def perform_scenario_analysis(base_params, scenarios, wtp_threshold=50000):
    """
    Perform scenario analysis with different parameter sets.
    
    Args:
        base_params: Dictionary of base case parameters
        scenarios: List of scenario dictionaries
        wtp_threshold: Willingness-to-pay threshold
        
    Returns:
        DataFrame with results for each scenario
    """
    results = []
    
    for scenario in scenarios:
        # Apply scenario adjustments to base parameters
        params = base_params.copy()
        params.update(scenario['adjustments'])
        
        # Calculate outcomes for each strategy in this scenario
        for strategy in ['ECT', 'IV-KA', 'PO-KA']:
            # Calculate cost and effect based on parameters
            # This is a simplified model - in practice, you would have more complex calculations
            base_cost = params.get('base_cost', 5000)
            base_effect = params.get('base_effect', 0.6)
            
            # Adjust cost and effect based on strategy
            if strategy == 'IV-KA':
                cost = base_cost * params.get('cost_iv_ka_multiplier', 1.0)
                effect = base_effect * params.get('effect_iv_ka_multiplier', 1.2)
            elif strategy == 'PO-KA':
                cost = base_cost * params.get('cost_po_ka_multiplier', 0.9)
                effect = base_effect * params.get('effect_po_ka_multiplier', 1.1)
            else:  # ECT
                cost = base_cost
                effect = base_effect
            
            # Apply scenario-specific adjustments
            if f'cost_{strategy.lower()}_adjustment' in params:
                cost = cost * params[f'cost_{strategy.lower()}_adjustment']
            if f'effect_{strategy.lower()}_adjustment' in params:
                effect = effect * params[f'effect_{strategy.lower()}_adjustment']
            
            # Calculate metrics
            nmb = effect * wtp_threshold - cost
            
            results.append({
                'scenario': scenario['name'],
                'strategy': strategy,
                'cost': cost,
                'effect': effect,
                'net_monetary_benefit': nmb,
                'wtp_threshold': wtp_threshold
            })
    
    return pd.DataFrame(results)

In [ ]:
# Define base case parameters
base_params = {
    'base_cost': 5000,
    'base_effect': 0.6,
    'cost_iv_ka_multiplier': 1.5,  # IV-KA costs 50% more than base
    'effect_iv_ka_multiplier': 1.33,  # IV-KA is 33% more effective
    'cost_po_ka_multiplier': 1.2,  # PO-KA costs 20% more than base
    'effect_po_ka_multiplier': 1.17  # PO-KA is 17% more effective
}

# Define scenarios
scenarios = [
    {
        'name': 'Base Case',
        'adjustments': {}
    },
    {
        'name': 'High Cost Environment',
        'adjustments': {
            'base_cost': 7000  # Higher base costs
        }
    },
    {
        'name': 'High Effectiveness',
        'adjustments': {
            'base_effect': 0.8  # Higher base effectiveness
        }
    },
    {
        'name': 'Favorable IV-KA',
        'adjustments': {
            'effect_iv_ka_multiplier': 1.5,  # IV-KA even more effective
            'cost_iv_ka_multiplier': 1.3   # IV-KA less costly
        }
    },
    {
        'name': 'Conservative WTP',
        'adjustments': {
            'wtp_threshold': 40000  # Lower WTP threshold
        }
    },
    {
        'name': 'Optimistic Market',
        'adjustments': {
            'cost_ect_adjustment': 0.95,  # Lower ECT costs
            'cost_iv_ka_adjustment': 0.9,  # Lower IV-KA costs
            'cost_po_ka_adjustment': 0.85, # Lower PO-KA costs
            'wtp_threshold': 60000  # Higher WTP threshold
        }
    }
]

# Perform scenario analysis
wtp_threshold = 50000
scenario_results = perform_scenario_analysis(base_params, scenarios, wtp_threshold)
print(scenario_results.head(10))

In [ ]:
# Visualize scenario analysis results
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# 1. Cost by scenario and strategy
cost_pivot = scenario_results.pivot_table(
    values='cost', 
    index='scenario', 
    columns='strategy', 
    aggfunc='mean'
)
sns.heatmap(cost_pivot, annot=True, fmt='.0f', cbar_kws={'label': 'Cost ($AUD)'}, ax=ax[0,0])
ax[0,0].set_title('Cost by Scenario and Strategy')

# 2. Effect by scenario and strategy
effect_pivot = scenario_results.pivot_table(
    values='effect', 
    index='scenario', 
    columns='strategy', 
    aggfunc='mean'
)
sns.heatmap(effect_pivot, annot=True, fmt='.3f', cbar_kws={'label': 'Effect (QALYs)'}, ax=ax[0,1])
ax[0,1].set_title('Effect by Scenario and Strategy')

# 3. Net Monetary Benefit by scenario and strategy
nmb_pivot = scenario_results.pivot_table(
    values='net_monetary_benefit', 
    index='scenario', 
    columns='strategy', 
    aggfunc='mean'
)
sns.heatmap(nmb_pivot, annot=True, fmt='.0f', cbar_kws={'label': 'NMB ($AUD)'}, ax=ax[1,0], center=0, cmap='RdBu_r')
ax[1,0].set_title('Net Monetary Benefit by Scenario and Strategy')

# 4. Scenario ranking
# Calculate top strategy for each scenario
top_strategies = []
for scenario in scenario_results['scenario'].unique():
    scenario_data = scenario_results[scenario_results['scenario'] == scenario]
    best_strategy = scenario_data.loc[scenario_data['net_monetary_benefit'].idxmax()]
    top_strategies.append({
        'scenario': scenario,
        'best_strategy': best_strategy['strategy'],
        'best_nmb': best_strategy['net_monetary_benefit']
    })

top_df = pd.DataFrame(top_strategies)
colors = ['blue' if s == 'ECT' else 'green' if s == 'IV-KA' else 'orange' for s in top_df['best_strategy']]
bars = ax[1,1].bar(range(len(top_df)), top_df['best_nmb'], color=colors, alpha=0.7)
ax[1,1].set_xlabel('Scenario')
ax[1,1].set_ylabel('Net Monetary Benefit ($AUD)')
ax[1,1].set_title('Best Strategy by Scenario')
ax[1,1].set_xticks(range(len(top_df)))
ax[1,1].set_xticklabels(top_df['scenario'], rotation=45, ha='right')

# Add strategy labels
for i, bar, strat in zip(range(len(bars)), bars, top_df['best_strategy']):
    height = bar.get_height()
    ax[1,1].text(bar.get_x() + bar.get_width()/2., height + max(top_df['best_nmb'])*0.01,
                 f'{strat}', ha='center', va='bottom', fontweight='bold')

ax[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create a tornado chart to show the sensitivity of the decision to different scenarios
plt.figure(figsize=(12, 8))

# Calculate the range of NMB for each strategy across scenarios
strategy_ranges = []
for strategy in ['ECT', 'IV-KA', 'PO-KA']:
    strategy_data = scenario_results[scenario_results['strategy'] == strategy]
    min_nmb = strategy_data['net_monetary_benefit'].min()
    max_nmb = strategy_data['net_monetary_benefit'].max()
    base_nmb = strategy_data[strategy_data['scenario'] == 'Base Case']['net_monetary_benefit'].iloc[0]
    
    strategy_ranges.append({
        'strategy': strategy,
        'min_nmb': min_nmb,
        'max_nmb': max_nmb,
        'base_nmb': base_nmb,
        'range': max_nmb - min_nmb
    })

ranges_df = pd.DataFrame(strategy_ranges)
ranges_df = ranges_df.sort_values('range', ascending=True)  # Sort by range for tornado effect

# Create the tornado chart
y_pos = np.arange(len(ranges_df))
plt.barh(y_pos, ranges_df['max_nmb'] - ranges_df['min_nmb'], 
         left=ranges_df['min_nmb'], 
         color=['blue' if s == 'ECT' else 'green' if s == 'IV-KA' else 'orange' for s in ranges_df['strategy']],
         alpha=0.7)

# Add base case markers
plt.scatter(ranges_df['base_nmb'], y_pos, color='red', s=100, zorder=5, label='Base Case')

plt.yticks(y_pos, ranges_df['strategy'])
plt.xlabel('Net Monetary Benefit Range ($AUD)')
plt.title('Sensitivity of Net Monetary Benefit to Scenario Assumptions')
plt.grid(True, axis='x', alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Calculate decision uncertainty across scenarios
print("Decision Consistency Across Scenarios")
print("="*50)

# Count how often each strategy is optimal
optimality_counts = {}
for scenario in scenario_results['scenario'].unique():
    scenario_data = scenario_results[scenario_results['scenario'] == scenario]
    best_strategy = scenario_data.loc[scenario_data['net_monetary_benefit'].idxmax(), 'strategy']
    
    optimality_counts[best_strategy] = optimality_counts.get(best_strategy, 0) + 1

for strategy, count in optimality_counts.items():
    print(f"{strategy} is optimal in {count} out of {len(scenario_results['scenario'].unique())} scenarios ({count/len(scenario_results['scenario'].unique()):.1%})")

print('\n' + "Scenario Analysis Summary:")
print("="*30)

for scenario in scenario_results['scenario'].unique():
    scenario_data = scenario_results[scenario_results['scenario'] == scenario]
    best_strategy = scenario_data.loc[scenario_data['net_monetary_benefit'].idxmax()]
    
    print(f"\n{scenario}:")
    print(f"  Best Strategy: {best_strategy['strategy']} (NMB: ${best_strategy['net_monetary_benefit']:,.0f})")
    
    # Show all strategies for comparison
    for _, row in scenario_data.iterrows():
        print(f"    {row['strategy']}: Cost=${row['cost']:,.0f}, Effect={row['effect']:.3f}, NMB=${row['net_monetary_benefit']:,.0f}")

In [ ]:
# Create a decision matrix for scenario robustness
plt.figure(figsize=(14, 8))

# Create a matrix showing which strategy is best in each scenario
all_scenarios = scenario_results['scenario'].unique()
all_strategies = ['ECT', 'IV-KA', 'PO-KA']

# Create a matrix for visualization
matrix = np.zeros((len(all_scenarios), len(all_strategies)))
strategy_map = {s: i for i, s in enumerate(all_strategies)}

# Fill the matrix with NMB values
for scenario_idx, scenario in enumerate(all_scenarios):
    scenario_data = scenario_results[scenario_results['scenario'] == scenario]
    for _, row in scenario_data.iterrows():
        strategy_idx = strategy_map[row['strategy']]
        matrix[scenario_idx, strategy_idx] = row['net_monetary_benefit']

# Create a heatmap showing which strategy is optimal
matrix_df = pd.DataFrame(matrix, index=all_scenarios, columns=all_strategies)

# Create a color map based on which strategy is best
best_strategy_matrix = np.zeros_like(matrix, dtype=int)
for i in range(len(matrix)):
    best_strategy_idx = np.argmax(matrix[i])
    best_strategy_matrix[i, best_strategy_idx] = 1

# Plot the matrix
sns.heatmap(best_strategy_matrix, 
            xticklabels=all_strategies, 
            yticklabels=all_scenarios,
            annot=matrix_df.applymap(lambda x: f'{x:,.0f}'), 
            fmt='',
            cbar=False,
            cmap='viridis',
            ax=plt.gca())

# Highlight the best strategy for each scenario
for i in range(len(all_scenarios)):
    best_strategy_idx = np.argmax(matrix[i])
    plt.gca().add_patch(plt.Rectangle((best_strategy_idx, i), 1, 1, fill=False, 
                                      edgecolor='red', lw=3))

plt.title('Scenario Analysis: Best Strategy by Scenario')
plt.xlabel('Strategy')
plt.ylabel('Scenario')
plt.tight_layout()
plt.show()

## Next Steps

1. Define more complex scenarios with multiple parameter changes
2. Incorporate correlation between parameters
3. Perform multi-way scenario analysis
4. Consider budget constraints and market access